In [ ]:
import os
from sentence_transformers import SentenceTransformer
import chromadb
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer
class EmbeddStoreChromaST:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name,device = "cpu")

    @classmethod
    def save_vectors_to_chromadb(cls, embeddings, documents, db_path):
        client = chromadb.PersistentClient(path=os.path.abspath(db_path))
        try:
            client.delete_collection("testdb10")
        except:
            pass
        collection = client.create_collection("testdb10")
    
        collection.add(embeddings=embeddings, documents=documents, ids=[f"id{i}" for i in range(1, len(embeddings) + 1)])
        print(f"ChromaDB saved successfully to {db_path}")

    def encode_text(self, text):
        embeddings = self.model.encode(text, convert_to_tensor=True)
        return embeddings.tolist()



with open('ribbon_global.txt', 'r', encoding='utf-8') as file:
    file_contents = file.read()

segmented_text = EmbeddStoreChromaST.segment_text(file_contents, chunk_size=50)

model = EmbeddStoreChromaST(model_name='sentence-transformers/all-mpnet-base-v2')

embeddings = model.encode_text(segmented_text)

database_path = f"./embedded_v2/path_1"
model.save_vectors_to_chromadb(embeddings, segmented_text, database_path)



In [1]:
import os
from sentence_transformers import SentenceTransformer
import chromadb
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer
import requests
from chromadb.utils import embedding_functions

class RetrivalChromaST:
    def __init__(self, model_name):
        self.model = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name)

    def encode_text(self, text):
        embeddings = self.model.encode(text, convert_to_tensor=True)
        return embeddings.tolist()

    @classmethod
    def load_chromadb(cls, db_path, user_input, embedder1):
        client = chromadb.PersistentClient(path=os.path.abspath(db_path))
        collection = client.get_collection("testdb10", embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name=embedder1))
        results = collection.query(
            query_texts=[user_input],
            n_results=2,
        )
        score = results["distances"][0]
        context_only = results["documents"][0]
        context_with_score = [[i, j] for i, j in zip(context_only, score)]
        return context_with_score

user_input = input("User input: ")


model_name = 'sentence-transformers/all-mpnet-base-v2'
model = RetrivalChromaST(model_name=model_name)
database_path1 = f"./embedded_v2/path_1"
similar_texts = RetrivalChromaST.load_chromadb(database_path1, user_input, embedder1=model_name)
print(similar_texts)
similar_texts1 = [i[0] for i in similar_texts]
final_context = " ".join(similar_texts1)
print("##################################################################################")

payload = {
    "message_content": user_input,
    "content": final_context,
    "company_name": "ribbon global",
    "domain": "global services"
}

response = requests.post("http://46.43.144.145:7010/rulebased_bot", json=payload)

print(response.json())


User input:  hi


ValueError: Collection testdb10 does not exist.

AttributeError: 'SentenceTransformerEmbeddingFunction' object has no attribute 'encode'